In [1]:
%cd ../data

/home/philippe/MolGenDocking/data


/home/philippe/miniconda3/envs/MolGen/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from huggingface_hub import HfApi

In [3]:
DATASET_NAMES = ["property_prediction","molgendata", "synthesis_tasks",]

In [4]:
import os
import time

api = HfApi()
for name in DATASET_NAMES[:]:
    os.system("rm {}.tar.gz".format(name))
    print("Deleted '{}', compressing again".format(name))
    t0 = time.time()
    os.system("tar -czf {}.tar.gz {}".format(name, name))
    t1 = time.time()
    print("Compressed '{}' in {:.2f}s".format(name, t1-t0))
    api.upload_file(
        path_or_fileobj="{}.tar.gz".format(name),
        path_in_repo="{}.tar.gz".format(name),
        repo_id="Franso/MolGenData",
        repo_type="dataset",
    )
    t2 = time.time()
    print("Uploaded '{}' in {:.2f}s".format(name, t2-t1))



Deleted 'property_prediction', compressing again
Compressed 'property_prediction' in 0.35s


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

No files have been modified since last commit. Skipping to prevent empty commit.


Uploaded 'property_prediction' in 0.88s
Deleted 'molgendata', compressing again
Compressed 'molgendata' in 77.21s


...

New Data Upload: |          |  0.00B /  0.00B            

Uploaded 'molgendata' in 6.72s
Deleted 'synthesis_tasks', compressing again
Compressed 'synthesis_tasks' in 2.82s


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Uploaded 'synthesis_tasks' in 1.71s


In [5]:
FILES = [
    "molgendata/test_data/test_prompts_ood_boxed.jsonl",
    "property_prediction/eval_prompts_boxed.jsonl",
    "synthesis_tasks/eval_prompts/eval_prompts_enamine.jsonl",
    "synthesis_tasks/eval_prompts/eval_prompts_chembl.jsonl"
]

In [6]:
from datasets import Dataset
import pandas as pd
from pathlib import Path
from mol_gen_docking.data.pydantic_dataset import read_jsonl

def iterable_from_dataset(dataset):
    for item in dataset:
        conv = item.conversations[0]
        yield {
            "id": conv.identifier,
            "message": {"role": conv.messages[0].role, "content": conv.messages[0].content},
        }

def to_hf_format(dataset):
    dict_dataset = {}
    for line in iterable_from_dataset(dataset):
        for k, v in line.items():
            if k not in dict_dataset:
                dict_dataset[k] = []
            dict_dataset[k].append(v)

    return Dataset.from_dict(dict_dataset)

In [7]:
DATASET_NAMES = ["property_prediction","molgendata", "synthesis_tasks",]

for name in DATASET_NAMES[:]:
    path = Path(name)
    # Upload training datasets
    for data_path in path.rglob("train_prompts*.jsonl"):
        print(name, data_path)
        dataset = read_jsonl(data_path)
        hf_dataset = to_hf_format(dataset)
        hf_dataset.push_to_hub(
            data_dir="{}_hf".format(name),
            repo_id="Franso/MolGenData",
            split="train"
        )

property_prediction property_prediction/train_prompts_boxed.jsonl


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

README.md:   0%|          | 0.00/523 [00:00<?, ?B/s]

In [8]:
DATASET_NAMES = ["property_prediction","molgendata", "synthesis_tasks",]
for name in DATASET_NAMES[:]:
    path = Path(name)
    # Upload training datasets
    for data_path in path.rglob("eval_prompts*.jsonl"):
        print(name, data_path)
        dataset = read_jsonl(data_path)
        hf_dataset = to_hf_format(dataset)
        hf_dataset.push_to_hub(
            data_dir="{}_hf".format(name),
            repo_id="Franso/MolGenData",
            split="validation"
        )

property_prediction property_prediction/eval_prompts_boxed.jsonl


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

README.md:   0%|          | 0.00/528 [00:00<?, ?B/s]